In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [59]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [60]:
max_quinzaine=24
max_area_lat=12
max_area_lon=31
number_grid=(max_area_lat+1)*(max_area_lon+1)
print number_grid

416


In [61]:
df['lat_area'] = df['lat_area'].astype(int)
df['lon_area'] = df['lon_area'].astype(int)

# Version 1

In [51]:
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [52]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week","area"] ).size()}).reset_index()

In [53]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head()

   Annee (0)  quinzaine_week  area  count lat_area lon_area
0       2008               3  4_20      2        4       20
1       2008               6  3_21      1        3       21
2       2008               6  4_18      1        4       18
3       2008               6  6_19      1        6       19
4       2008               7  5_16      1        5       16


In [54]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
Colyears=np.repeat(uniqueYears, 24*(max_area_lon+1)*(max_area_lat+1))

TwoweekRepeat=np.repeat(np.linspace(1, max_quinzaine, num=max_quinzaine), (max_area_lon+1)*(max_area_lat+1))
ColTwoweek=np.tile(TwoweekRepeat, len(uniqueYears))

latRepeat=np.linspace(0, max_area_lat,num=max_area_lat+1)
ColLatArea=np.repeat(latRepeat, max_area_lon+1)
ColLatArea=np.tile(ColLatArea, max_quinzaine)
ColLatArea=np.tile(ColLatArea, len(uniqueYears))

#print ColLatArea

lonRepeat=np.tile(np.linspace(0, max_area_lon, num=max_area_lon+1),max_area_lat+1)
ColLonArea=np.tile(lonRepeat, max_quinzaine)
ColLonArea=np.tile(ColLonArea, len(uniqueYears))

print ColLonArea

count=np.repeat(0, len(Colyears))

print len(Colyears)
print len(ColTwoweek)
print len(ColLatArea)
print len(ColLonArea)


[  0.   1.   2. ...,  29.  30.  31.]
69888
69888
69888
69888


## Fill the matrice

In [55]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     '2_week': ColTwoweek, 
     'lat_area': ColLatArea,
     'lon_area': ColLonArea,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal['lat_area']=dfFinal['lat_area'].astype(int).astype('str')
dfFinal['lon_area']=dfFinal['lon_area'].astype(int).astype('str')

print dfFinal.head()

   2_week  Years  count lat_area lon_area
0       1   2008      0        0        0
1       1   2008      0        0        1
2       1   2008      0        0        2
3       1   2008      0        0        3
4       1   2008      0        0        4


In [56]:
for index, row in dfAgregateTotal.iterrows():
    if index%100==0:
        print index
    #print row['Annee (0)'], row['quinzaine_week'], row['count'] ,  row['lat_area'] , row['lon_area'] 
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) & (dfFinal['lat_area']==row['lat_area']) & (dfFinal['lon_area']==row['lon_area'] )& (dfFinal['2_week']==row['quinzaine_week'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal.set_value(idx.index.values, 'count',row['count'])

0
100
200
300


In [38]:
dfFinal['count'].sum()

421

## Write matrice in a file

In [57]:
dfFinal.to_csv("../data/aden_golfe_matrix_area.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 2

In [62]:
dfAgregateTotal2=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week"] ).size()}).reset_index()
dfAgregateTotal2.head()

,Annee (0),quinzaine_week,count
0,2008,3,2
1,2008,6,3
2,2008,7,3
3,2008,8,3
4,2008,10,6


In [63]:
unique_years=dfAgregateTotal2['Annee (0)'].unique()

#unique_type=dfAgregateTotal2['type d\'attaque(0)'].unique()

print unique_years
#print unique_type

[2008 2009 2010 2011 2012 2013 2014]


In [42]:
col_years=np.repeat(unique_years, max_quinzaine)
print len(col_years)

two_week_repeat=np.linspace(1, max_quinzaine, num=max_quinzaine)
#print np.repeat(two_week_repeat,4)
col_two_week=np.tile(two_week_repeat, len(unique_years))
print len(col_two_week)

col_count=np.repeat(0, len(col_two_week))


dic2 = {'Years': col_years, 
     '2_week': col_two_week, 
     'count': col_count#,
     #'type_attack': col_type
      }

dfFinal_2= pd.DataFrame(dic2)


168
168


## Matrix empty

In [43]:
dfFinal_2.head()

,2_week,Years,count
0,1,2008,0
1,2,2008,0
2,3,2008,0
3,4,2008,0
4,5,2008,0


# Fill matrice

In [44]:
for index, row in dfAgregateTotal2.iterrows():

    print 'Years '+str(row['Annee (0)']),'2_week : '+str( row['quinzaine_week']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfFinal_2[(dfFinal_2['Years']==row['Annee (0)']) & (dfFinal_2['2_week']==row['quinzaine_week'])]
    print 'index : '+str(idx.index.values)
    print 'count : '+str(row['count'])
    dfFinal_2.set_value(idx.index.values, 'count',row['count'])

Years 2008.0 2_week : 3.0 count : 2.0
index : [2]
count : 2.0
Years 2008.0 2_week : 6.0 count : 3.0
index : [5]
count : 3.0
Years 2008.0 2_week : 7.0 count : 3.0
index : [6]
count : 3.0
Years 2008.0 2_week : 8.0 count : 3.0
index : [7]
count : 3.0
Years 2008.0 2_week : 10.0 count : 6.0
index : [9]
count : 6.0
Years 2008.0 2_week : 11.0 count : 1.0
index : [10]
count : 1.0
Years 2008.0 2_week : 12.0 count : 1.0
index : [11]
count : 1.0
Years 2008.0 2_week : 13.0 count : 1.0
index : [12]
count : 1.0
Years 2008.0 2_week : 14.0 count : 3.0
index : [13]
count : 3.0
Years 2008.0 2_week : 15.0 count : 2.0
index : [14]
count : 2.0
Years 2008.0 2_week : 16.0 count : 7.0
index : [15]
count : 7.0
Years 2008.0 2_week : 17.0 count : 14.0
index : [16]
count : 14.0
Years 2008.0 2_week : 18.0 count : 5.0
index : [17]
count : 5.0
Years 2008.0 2_week : 19.0 count : 9.0
index : [18]
count : 9.0
Years 2008.0 2_week : 20.0 count : 7.0
index : [19]
count : 7.0
Years 2008.0 2_week : 21.0 count : 8.0
index : 

In [45]:
#binary=label_binarize(col_type, classes=unique_type)
#dfFinal_2[unique_type[0]]=binary[:,0]
#dfFinal_2[unique_type[1]]=binary[:,1]
#dfFinal_2[unique_type[2]]=binary[:,2]
#dfFinal_2[unique_type[3]]=binary[:,3]

dfFinal_2['full_years'] =dfFinal_2.apply(lambda row: row['Years']+float(row['2_week']/24), axis=1)
print dfFinal_2

     2_week  Years  count   full_years
0         1   2008      0  2008.041667
1         2   2008      0  2008.083333
2         3   2008      2  2008.125000
3         4   2008      0  2008.166667
4         5   2008      0  2008.208333
5         6   2008      3  2008.250000
6         7   2008      3  2008.291667
7         8   2008      3  2008.333333
8         9   2008      0  2008.375000
9        10   2008      6  2008.416667
10       11   2008      1  2008.458333
11       12   2008      1  2008.500000
12       13   2008      1  2008.541667
13       14   2008      3  2008.583333
14       15   2008      2  2008.625000
15       16   2008      7  2008.666667
16       17   2008     14  2008.708333
17       18   2008      5  2008.750000
18       19   2008      9  2008.791667
19       20   2008      7  2008.833333
20       21   2008      8  2008.875000
21       22   2008      8  2008.916667
22       23   2008      4  2008.958333
23       24   2008      6  2009.000000
24        1   2009     17

In [46]:
dfFinal_2.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)

In [64]:
dfAgregateTotal2.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)